In [107]:
import numpy as np
import pandas as pd
import collections
from difflib import SequenceMatcher

In [108]:

#constants
K = 5

In [109]:
#functions
def edit_distance(string1:str,string2:str):
    return SequenceMatcher(None,a=string1,b = string2).ratio()


In [110]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

# Pre-Processing

In [111]:

#data reading
data = {}
with open('data.txt','r') as data_set:
    for line in data_set.readlines():
        if line[0] == '>':
            index = line[1:].rstrip('\n')
            data[index] = ""
        else:
            data[index] = data[index] + line.rstrip('\n')


In [112]:
# calculate the distance matrix
distance_matrix = pd.DataFrame(columns=list(data.keys()),index=list(data.keys()))

In [113]:
def distance(acids:pd.Series,acid:str):
  # returns distance between 'acid' and all the elements of 'acids'
  return acids.reset_index().apply(lambda x:levenshteinDistance(x['index'],acid),axis=1)

In [114]:
distance_matrix = distance_matrix.apply(lambda x:distance(x,x.name),axis=1)

In [115]:
distance_matrix.columns = list(data.keys())

# K-mediods

In [116]:
import time

t1 = time.time()
cluster_centre_indices = np.random.randint(low=0,high = len(data.keys()),size=K) 

print("initial cluster centres ",cluster_centre_indices)

def choose_closest_cluster(x):
  # selects the closest cluster to x
  return np.argmin(x[cluster_centre_indices])

a = distance_matrix.apply(lambda x:choose_closest_cluster(x),axis=1)

def check_loss(x):
  # returns the loss of a given cluster distribution
  return levenshteinDistance(x['index'],x[0])


error = sum(a.reset_index().apply(lambda x:check_loss(x),axis=1))
print("intial error",error)

iteration = 1

while True:
  #swapping
    print(iteration)
    flag = 0
    temp = cluster_centre_indices
    for s_idx in cluster_centre_indices:
        for u_idx in range(len(data.keys())):
            if u_idx not in cluster_centre_indices:
        #unselected idx
                cluster_centre_indices[cluster_centre_indices == s_idx] = u_idx
        # let each point choos its closest cluster
                a = distance_matrix.apply(lambda x:choose_closest_cluster(x),axis=1)
        # find the new error
                new_error = sum(a.reset_index().apply(lambda x:check_loss(x),axis=1))
                if new_error >= error:
                  # the previous configuration is better,hence we revert back
                  cluster_centre_indices[cluster_centre_indices == u_idx] = s_idx
                else:
          # the new configuration is better
                    error = new_error
                    flag = 1
                    print("Swapped {} with {}".format(s_idx,u_idx))
                    print("Present Cluster centres ",cluster_centre_indices)
                    print("present error ",error)
                    continue
       
    if flag == 0:
        print("No change in {} iteration.\nError{}".format(iteration,error))
        break
    else:
        iteration += 1
        print("*******")

print(time.time() - t1)  

initial cluster centres  [135 428 556 445 394]
intial error 3393
1


/home/shishir/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:51: FutureWarning:

'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.



KeyboardInterrupt: 

# **Agglomerative**

In [104]:
linkage_type = 'min' # min , max ,avg

In [105]:
distance_array  = distance_matrix.values

#functions
def cluster_distance(c1:list,c2:list,link_type:str):
  # returns the distance between two clusters given the type of linkage
    d = distance_array[np.ix_(c1,c2)]
    if link_type == 'min':
        return d.min()
    elif link_type == 'max':
        return d.max()
    elif link_type == 'avg':
        return d.mean()
  
    

In [106]:
from itertools import combinations
import time
t1 = time.time()
names = list(data.keys())
clusters = [[names.index(i)] for i in names] # intial indiviual clusters
cluster_points = {frozenset(k):(clusters.index(k),0) for k in clusters} # viz
lines = []
level = 0
while True:
    level += 1
    N = len(clusters)

    print("Level: ",level)
    print("Number of clusters: ",N)

    if N == 1:
        break # all clusters are combined into one

    # get all combinations of clusters.
    pairs = list(combinations(clusters,2))
    # find pair with minimmum inter cluster distance to merge
    min_pair = min(pairs,key = lambda pair:cluster_distance(pair[0],pair[1],linkage_type))

    merge_cluster1 = min_pair[0]
    merge_cluster2 = min_pair[1]
    new_cluster= merge_cluster1 + merge_cluster2
    # add the merged cluster while removing the previous clusters
    clusters.remove(merge_cluster1)
    clusters.remove(merge_cluster2)
    clusters.append(new_cluster)

print(time.time() - t1)  

Level:  1
Number of clusters:  570
Level:  2
Number of clusters:  569
Level:  3
Number of clusters:  568
Level:  4
Number of clusters:  567
Level:  5
Number of clusters:  566
Level:  6
Number of clusters:  565
Level:  7
Number of clusters:  564
Level:  8
Number of clusters:  563
Level:  9
Number of clusters:  562
Level:  10
Number of clusters:  561
Level:  11
Number of clusters:  560
Level:  12
Number of clusters:  559
Level:  13
Number of clusters:  558
Level:  14
Number of clusters:  557
Level:  15
Number of clusters:  556
Level:  16
Number of clusters:  555
Level:  17
Number of clusters:  554
Level:  18
Number of clusters:  553
Level:  19
Number of clusters:  552
Level:  20
Number of clusters:  551
Level:  21
Number of clusters:  550
Level:  22
Number of clusters:  549
Level:  23
Number of clusters:  548
Level:  24
Number of clusters:  547
Level:  25
Number of clusters:  546
Level:  26
Number of clusters:  545
Level:  27
Number of clusters:  544
Level:  28
Number of clusters:  543
L

Level:  226
Number of clusters:  345
Level:  227
Number of clusters:  344
Level:  228
Number of clusters:  343
Level:  229
Number of clusters:  342
Level:  230
Number of clusters:  341
Level:  231
Number of clusters:  340
Level:  232
Number of clusters:  339
Level:  233
Number of clusters:  338
Level:  234
Number of clusters:  337
Level:  235
Number of clusters:  336
Level:  236
Number of clusters:  335
Level:  237
Number of clusters:  334
Level:  238
Number of clusters:  333
Level:  239
Number of clusters:  332
Level:  240
Number of clusters:  331
Level:  241
Number of clusters:  330
Level:  242
Number of clusters:  329
Level:  243
Number of clusters:  328
Level:  244
Number of clusters:  327
Level:  245
Number of clusters:  326
Level:  246
Number of clusters:  325
Level:  247
Number of clusters:  324
Level:  248
Number of clusters:  323
Level:  249
Number of clusters:  322
Level:  250
Number of clusters:  321
Level:  251
Number of clusters:  320
Level:  252
Number of clusters:  319
L

Level:  448
Number of clusters:  123
Level:  449
Number of clusters:  122
Level:  450
Number of clusters:  121
Level:  451
Number of clusters:  120
Level:  452
Number of clusters:  119
Level:  453
Number of clusters:  118
Level:  454
Number of clusters:  117
Level:  455
Number of clusters:  116
Level:  456
Number of clusters:  115
Level:  457
Number of clusters:  114
Level:  458
Number of clusters:  113
Level:  459
Number of clusters:  112
Level:  460
Number of clusters:  111
Level:  461
Number of clusters:  110
Level:  462
Number of clusters:  109
Level:  463
Number of clusters:  108
Level:  464
Number of clusters:  107
Level:  465
Number of clusters:  106
Level:  466
Number of clusters:  105
Level:  467
Number of clusters:  104
Level:  468
Number of clusters:  103
Level:  469
Number of clusters:  102
Level:  470
Number of clusters:  101
Level:  471
Number of clusters:  100
Level:  472
Number of clusters:  99
Level:  473
Number of clusters:  98
Level:  474
Number of clusters:  97
Leve

# **DIANA**

In [42]:
#functions

def most_disimilar(cluster:list):
  # returns the data point with highest average distance
  return np.argmax(distance_matrix.loc[cluster,cluster].mean())

def avg_distance(cluster:list):
  # returns the average inter-distance between clusters.
  return distance_matrix.loc[cluster,cluster].values.mean()

In [45]:
t1 = time.time()
total = list(data.keys())
number = len(total)
clusters = [total]

#viz
cluster_points = {frozenset(clusters[0]):(len(clusters[0])/2,5)}
lines = []

#no of clusters to be formed
K = number
k = 1

while True:
    N = len(clusters)
    if N == len(data.keys()):
    #clusters are completely split
        break

    print(N)

    # break into clusters
    if 1:
        k += 1
        
        if k > K:
          # required clusters are broken
          break
        # find most heterogenus cluster
        to_split_cluster = max(clusters,key = lambda x: avg_distance(x))
        # cant break the cluster anymore
        if len(to_split_cluster) == 1:
            break
        # find the most disimilar point in the cluster
        odd_one = most_disimilar(to_split_cluster)
        # c1 is the splinter cell
        c1 = [odd_one]
        c2 = [c for c in to_split_cluster if c != odd_one]
        for element in c2:
          #if closer to c1 send to c1
          if distance_matrix.loc[element,c1].mean() < distance_matrix.loc[element,c2].mean():
            #send to c1
            c1.append(element)
            c2.remove(element)

        clusters.remove(to_split_cluster)
        clusters.append(c1)
        clusters.append(c2)

        print(time.time() - t1)

1
0.972959041595459
2
2.0668797492980957
3
2.1185264587402344
4
3.0230298042297363
5
3.0641655921936035
6
3.1006929874420166
7
3.1332130432128906
8
3.832152843475342
9
4.420447826385498
10
4.947223424911499
11
4.982079029083252
12
5.023125886917114
13
5.427088499069214
14
5.770724773406982
15
5.809354305267334
16
6.081063508987427
17
6.150594711303711
18
6.485618829727173
19
6.751344680786133
20
6.786369562149048
21
6.858338832855225
22
6.919156312942505
23
7.06666374206543
24
7.109860897064209
25
7.316349506378174
26
7.529371023178101
27
7.67929220199585
28
7.811595916748047
29
7.864260673522949
30
8.026193141937256
31
8.173568725585938
32
8.262914180755615
33
8.339323282241821
34
8.44943380355835
35
8.576712608337402
36
8.677106618881226
37
8.835193872451782
38
9.004045963287354
39
9.116378545761108
40
9.250539779663086
41
9.315414667129517
42
9.412435054779053
43
9.467699527740479
44
9.570488691329956
45
9.682780981063843
46
9.781553268432617
47
9.977176666259766
48
10.1728322505950

90.05823826789856
375
90.44873380661011
376
90.82744646072388
377
91.20306944847107
378
91.57965922355652
379
92.01241421699524
380
92.39271402359009
381
92.77017092704773
382
93.15393090248108
383
93.53240489959717
384
93.92976593971252
385
94.31902647018433
386
94.70509886741638
387
95.09237480163574
388
95.48034238815308
389
95.87434697151184
390
96.2619218826294
391
96.65003180503845
392
97.05104112625122
393
97.44049572944641
394
97.83660912513733
395
98.23914885520935
396
98.63002824783325
397
99.02662992477417
398
99.4220678806305
399
99.82221508026123
400
100.22361445426941
401
100.62070536613464
402
101.02702069282532
403
101.4277446269989
404
101.83558559417725
405
102.23589658737183
406
102.64161372184753
407
103.05003929138184
408
103.45302677154541
409
103.85565400123596
410
104.25874757766724
411
104.66304659843445
412
105.0762209892273
413
105.47849893569946
414
105.92026376724243
415
106.33972787857056
416
106.76161861419678
417
107.18892121315002
418
107.60394072532654

# Visualization

In [0]:
import plotly.plotly as py
import plotly.figure_factory as ff

D = linkage(X, method='average')

den = dendrogram(D)
plt.title('Dendrogram for the clustering of the dataset on three different varieties of wine')
plt.xlabel('Number of Instance')
plt.ylabel('Euclidean distance in the space with 13 dimensions');
plt.show()